# Individual-level genotype-expression data generation

## Goal
This pipeline aims to extract individual-level genotype-expression data in preparation for cis-eQTL fine-mapping.

## Overview of methods
Specifically, we are to organize the genotype, expression, and covariates data into a gene-wise form, i.e., a data file per gene. Besides, we need the expression data 'perpendicular' to covariates, which will be used for regression with genotype, in instrument variable idea.

## Details of methods
covariates (`Z`): This is independent of genes, so this is the same among files and can be directly copied from the original covariates data.

expression (`y`): As `grep` is too time-consuming, we construct `gene:line_number` as preprocessing. For each gene, the `line_number` is the line numbers of the gene in the #{tissues} expression data files. As a result, to extract the expression of each gene, it is only the job of reading the lines in the files, and combining the lines together.

genotype (`X`): Obtain TSS of all genes as preprocessing. Once querying a gene, take the TSS of the gene, extracting SNPs within a predefined flanking region around the TSS and treating their genotypes as X.

expression 'perpendicular' to covariates (`y_res`): The part of expression data that is independent from covariates.

### Crystallized plan for generating y with the main step `gene:line_number`

1) extract the column of ensembl_gene_id in the 49 expression files by `pandas`

2) assign `gene:line_number` for each gene in each tissue based on the extracted column just now

3) for each gene, extract the row of expression of the gene with sample names in each tissue using `sed`

4) that's it!

# Documentation

## Input
genotype data all in a file and expression and covariates data organized tissue-wise.

- genotype: .vcf.gz

```
    #CHROM  POS     ID      REF     ALT     QUAL    FILTER  INFO    FORMAT  GTEX-1117F      GTEX-111CU      GTEX-111FC      GTEX-111VG      GTEX-111YS      GTEX-1122O
    chr1    13526   chr1_13526_C_T_b38      C       T       .       PASS    AN=1676;AF=0.000596659;AC=1     GT      0|0     0|0     0|0     0|0     0|0     0|0
```    

- expression: .bed.gz
```
    #chr    start   end     gene_id GTEX-1117F      GTEX-111CU      GTEX-111FC      GTEX-111VG      GTEX-111YS      GTEX-1122O      GTEX-1128S
    chr1    29552   29553   ENSG00000227232.5       1.3135329173730264      -0.9007944960568992     -0.29268956046586164    -0.7324113622418431     -0.27475411245874887    -0.6990255198601908     0.18188866123299216
```

- covariates: .txt
```
    ID      GTEX-1117F      GTEX-111CU      GTEX-111FC      GTEX-111VG      GTEX-111YS
    PC1     -0.0867  0.0107  0.0099  0.0144  0.0154
    PC2     -0.0132 -0.0026 -0.0050 -0.0081 -0.0093
```    

## Output
An RDS file containing genotype specific for each gene and expression and covariates of each gene.


In [1]:
sos run Multi_tissues.ipynb -h

usage: sos run Multi_tissues.ipynb [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  get_gene_meta
  get_sample_names
  preprocess
  extract

Global Workflow Options:
  --cwd output (as path)
  --expression-data  paths(glob('/project2/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/eqtl/GTEx_Analysis_v8_eQTL_expression_matrices/*.gz'))

  --genotype-data /project2/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/genotypes/WGS/variant_calls/GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze.SHAPEIT2_phased.vcf.gz (as path)
  --covariates-data  paths(glob('/project2/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/eqtl/GTEx_Analysis_v8_eQTL_covariates/*.txt'))

  --gene-id-fil

## Example

**You can edit and change the following bash variable.** First edit and run the following bash variable.
```
work_dir=/scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise
```
Then run as follows:

```
cd $work_dir
sos run Multi_tissues.ipynb preprocess
sos run Multi_tissues.ipynb extract
```

## Minimal working example


If you want to only extract a self-defined list, eg, first 10 genes from the gene_id_file, 

```
work_dir=/scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise
cd $work_dir
sos run Multi_tissues.ipynb preprocess
head -10 output/ensembl_gene_id.txt > /tmp/test.txt
sos run Multi_tissues.ipynb extract --gene-id-file /tmp/test.txt
```

In [ ]:
[global]
from glob import glob
parameter: cwd = path('./output')
parameter: expression_data = paths(glob('/project2/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/eqtl/GTEx_Analysis_v8_eQTL_expression_matrices/*.gz'))
parameter: genotype_data = path('/project2/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/genotypes/WGS/variant_calls/GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze.SHAPEIT2_phased.vcf.gz')
parameter: covariates_data = paths(glob('/project2/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/eqtl/GTEx_Analysis_v8_eQTL_covariates/*.txt'))
parameter: gene_id_file = path(f"{cwd:a}/ensembl_gene_id.txt")

## Preprocessing

In [ ]:
# get gene-line mappping
[get_gene_meta_1]
input: expression_data, group_by = 1
output: f"{cwd:a}/{_input:bnn}.gene_line.json"
python3: expand = '${ }'
    import pandas as pd
    gene_id = pd.read_csv(${_input:r}, sep='\t', compression='gzip', header=0, skip_blank_lines=False, usecols = [3])
    gene_linenum = dict([(x,y+1) for y, x in enumerate(gene_id['gene_id'].tolist())])
    import json
    with open(${_output:r}, 'w') as outfile:
        json.dump({${_input:r}:gene_linenum}, outfile)

# TSS for genes
[get_gene_meta_2]
input: expression_data
output: f"{gene_id_file:an}.TSS.txt"
bash: expand = '${ }', workdir = cwd
    zless ${_input} | awk '{print $1"\t"$2"\t"$3"\t"$4}' | sort -u | sed 's/^...//' > ${_output}
    
# get all gene names
[get_gene_meta_3]
output: gene_id_file
bash: expand = '${ }', workdir = cwd
    awk '{print $4}' ${_input} | sed 1d | sort -u > ${_output}

In [ ]:
[get_sample_names]
depends: executable('bcftools')
input: genotype_data
output: f"{cwd:a}/samplenames_X.txt"
bash: expand=True
    bcftools query -l {_input} > {_output}
    
[preprocess]
sos_run('get_gene_meta + get_sample_names')

## Data extraction

In [2]:
[extract]
depends: executable('tabix'), R_library("data.table"), R_library('rjson'), R_library('tools')
genes = open(gene_id_file).read().splitlines()
input: for_each = 'genes'
output: f"{cwd:a}/{_genes}.Multi_Tissues.RDS"

task:

bash: expand = '${ }', workdir = cwd
    # X
    tabix -f ${genotype_data} `awk '$4 ~ /${_genes}/ {print $0}' ${gene_id_file:an}.TSS.txt | head -n 1 | awk '{$3=$2+1000000} {$2=$2-1000000} {print "chr"$1":"$2"-"$3}'` \
        | sed -e 's/0|0/0/g' -e 's/0|1/1/g' -e 's/1|0/1/g' -e 's/1|1/2/g' > ${_genes}_genotype.txt # still remain a potential problem of multi-allelic case.

R: expand = "${ }", workdir = cwd
    # Z
    Z <- lapply(c(${covariates_data:r,}), function(x) t(as.matrix(read.table(file = x, header = TRUE, sep = '\t', quote = "", row.names = 1))))
    for (i in 1:length(Z)) {
        rownames(Z[[i]]) <- gsub(pattern = "GTEX.", replacement = "GTEX-", x = rownames(Z[[i]])) # the format of sample names Z was changed somehow, from "GTEX-*" to "GTEX.*", so we need to convert it back.
        names(Z)[[i]] <- strsplit(x = c(${covariates_data:br,})[i], split = "[.]")[[1]][1] # add tissue names, so as to match with those of y
    }
    
    # y

    filenames_json <- list.files(path = ${cwd:ar}, pattern = "*.json$", full.names = TRUE)
    line_numbers <- lapply(filenames_json, function(x) rjson::fromJSON(file=x))
  
    extract_y <- function(y_file, line) {
        if (is.null(line)) return(NULL)
        # a sed version here
        # cmd <- paste0("zcat ", y_file, " | sed '", line, "q;d'")
        # yi <- data.table::fread(cmd=cmd)
        yi <- data.table::fread(file = y_file, skip = line - 1, nrows = 1)
        samplenames_yi <- data.table::fread(file = y_file, skip = 0, nrows = 1)
        colnames(yi) <- colnames(samplenames_yi)
        yi <- t(as.matrix(yi))
        yi <- yi[-1:-4, , drop = FALSE]
        # colnames(yi) <- file_path_sans_ext(file_path_sans_ext(basename(y_file))) # y_file:bnn, add tissue names
        return(yi)
    }
    
    y <- lapply(line_numbers, function(x) extract_y(names(x), x[[1]][["${_genes}"]]))
    for (i in 1:length(y)) {
        names(y)[[i]] <- strsplit(x = basename(names(line_numbers[[i]])), split = '[.]')[[1]][1]
    }
    y[sapply(y, is.null)] <- NULL
    
    ## tissue names matching between y and Z (y as reference)
    
    tissuenamesmatching <- function(x, reference) {
        x[match(names(reference), names(x))] # match list names
    }
    
    Z <- tissuenamesmatching(x = Z, reference = y)
    
    ## sample names matching between y and Z (Z as reference)
    
    samplenamesmatching <- function(x, reference) {
        lapply(1:length(reference), function(i) x[[i]][match(rownames(reference[[i]]), rownames(x[[i]])), , drop = FALSE]) # match sample names within each dataframe
    }
    
    y <- samplenamesmatching(x = y, reference = Z)
    
    # X
    X <- read.csv(file = "${cwd:a}/${_genes}_genotype.txt", sep = '\t', header = FALSE, row.names = 3, stringsAsFactors = FALSE)
    X <- X[,-(1:8)]
    samplenames_X <- scan(file = "${cwd:a}/samplenames_X.txt", what = character(), quote = "")
    colnames(X) <- samplenames_X
    X <- as.matrix(x = X)
    X <- t(X)
    
    # y_res
    y_res <- lapply(1:length(Z), function(i) .lm.fit(x = Z[[i]], y = y[[i]])$residuals)

    # save
    saveRDS(object = list(X = X, y = y, Z = Z, y_res = y_res), file = ${_output:r})
  
bash: expand = '${ }', workdir = cwd
    # remove intermediate files
    rm ${_genes}_genotype.txt